# Morgan指纹（圆形指纹）——AllChem.GetMorganFingerprint函数

扩展连通性指纹（Extended-Connectivity Fingerprints，ECFPs）是一种圆形拓扑指纹，可用于分子表示、相似性搜索、构效关系建模。它在药物设计中有诸多应用，是最受欢迎的相似性比对工具

ECFPs在ChemAxon中有如下两种表示方式
* 1.整数标识列（list of integer identifiers）
ECFPs可以通过一组不定长的整数标识符来表示，这是最原始和最精确的表示方法

* 2.定长比特串（fixed-length bit string）
传统的二进制分子表示方法是通过固定长度的比特串实现的。而ECFPs也可以通过“折叠”操作来形成定长比特串，也就是将底层的整数标识列压缩成一个更短长度的定长比特串（就是一个向量，常见的长度有1024,2048等）

## 生成流程

1.初始化原子标识符

- ECFP生成时，首先对给定分子的每一个非氢原子分配一个初始整数标识符。该标识符是通过把原子的属性（例如原子序号、连接数量等）打包，再经哈希函数转变成一个整数变量而得到，包含了相应原子的化学信息。
- 需要考虑的原子属性是ECFPs中一个重要的参数，可以完全由自己设置。

2.标识符的迭代更新

- 初始化后，会进行一系列迭代操作，将某一原子的初始标识符与邻近原子的标识符合并，直到到达设置的半径为止。每一轮迭代都会捕捉距中心原子越来越远的原子信息，最终经哈希运算，编码成为一个整数值，这些整数值合并形成一个整数标识列。

- 这种迭代更新的过程基于一种[**摩根算法（Morgan algorithm）**](https://doi.org/10.1021/c160017a018)

<img src=".\data\image\Morgan_fig1.png" alt="Morgan_atom" style="zoom:50%;" />

3.标识符去重

* 在最终生成指纹的过程中，会去除等价的原子标识符。等价意味着他们包含一组相同的键，或他们有相同的哈希值。在这步操作中，如果标识符的频数需要保留（ECFC模式），相同标识符出现的次数将会被保存

<img src=".\data\image\Morgan_fig2.png" alt="Morgan_atom" style="zoom:100%;" />
<img src=".\data\image\Morgan_fig3.png" alt="Morgan_atom" style="zoom:100%;" />

最终得到的是分子的定长比特串形式

## 代码示例

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [51]:
ms = [Chem.MolFromSmiles('CCO'), Chem.MolFromSmiles('COC')]
# molecule = Chem.MolFromSmiles(SMILES)
Drug_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=8) for x in ms]
# 这里为了方便后边的快速计算，将参数设置为8，一般实际使用时设置为1024或2048

In [52]:
fps = Drug_fingerprints[0]
print(fps.GetNumBits())
print(fps.ToBitString())

8
11100011


# DrugBank中药物相似性计算——DataStructs.TanimotoSimilarity函数

文献综述[willett1998.pdf](./book/willett1998.pdf)中讲述了**谷本系数（the Tanimoto coefficient）一直是基于片段的化学相似性研究的首选测量方法**

## the Tanimoto coefficient

![Tanimoto_coef](./data/image/Tanimoto_coef.png)

## 参数

对于以包含 n 个二进制值（例如指纹）的向量 XA 和 XB 为特征的对象 A 和 B

<img src=".\data\image\Tanimoto_fig1.png" style="zoom:50%;" />

这里只讲固定长度比特串的两个向量之间的算法，更多详细内容请阅读文献[willett1998.pdf](/data/book/willett1998.pdf)

In [53]:
print(Drug_fingerprints[0].ToBitString())
print(Drug_fingerprints[1].ToBitString())
print(DataStructs.TanimotoSimilarity( Drug_fingerprints[0],  Drug_fingerprints[1]))

11100011
01000001
0.4


In [54]:
a = 5 # 向量A中比特值为1的数量
b = 2 # 向量B中比特值为1的数量
c = 2 # 向量A与B中比特值都为1的数量
s = c/(a+b-c)
s

0.4

可以验证计算使用的公式,讲前边的参数改为12,1024，并得到新的a,b,c代入计算,结果相同

# 参考资料

[Morgan_Algorithm1965.pdf](./book/Morgan_Algorithm.pdf)

[willett1998.pdf](./book/willett1998.pdf)

[butina1999.pdf](./book/butina1999.pdf)

[【3.2.11】扩展连通性指纹（Extended Connectivity Fingerprints，ECFPs）原理介绍 - Sam' Note (qinqianshan.com)](https://qinqianshan.com/biology/chemical/ecfps/)